In [1]:
import pandas as pd
import numpy as np

In [2]:
title_akas = pd.read_csv('title_akas.tsv', sep = '\t',  low_memory = False)
title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [3]:
title_rating = pd.read_csv('title_ratings.tsv', sep = '\t',  low_memory = False)
title_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1684
1,tt0000002,6.0,207
2,tt0000003,6.5,1421
3,tt0000004,6.1,121
4,tt0000005,6.1,2221


In [4]:
title_basics = pd.read_csv('title_basics.tsv', sep = '\t',  low_memory = False)
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
title_rating = pd.read_csv('title_ratings.tsv', sep = '\t',  low_memory = False)
title_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1684
1,tt0000002,6.0,207
2,tt0000003,6.5,1421
3,tt0000004,6.1,121
4,tt0000005,6.1,2221


In [6]:
title_crew = pd.read_csv('title_crew.tsv', sep = '\t',  low_memory = False)
title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [57]:
movie = title_basics[title_basics['titleType'] == 'movie'].copy()
#movie.drop(['originalTitle', 'isAdult', 'endYear', 'runtimeMinutes'], axis = 1, inplace = True)

title_rating['averageRating'] = title_rating.averageRating.astype(object).copy()
title_rating['numVotes'] = title_rating.numVotes.astype(object).copy()

movie = movie.join(title_rating.set_index('tconst'), on = ['tconst'])
#movie = movie[movie['numVotes'] > 10000].copy()
#movie = movie[movie['averageRating'] > 3.0].copy()
movie = movie[movie['startYear'].str.isdigit()].copy()

movie['startYear'] = movie['startYear'].astype(int)

movie = movie[movie['startYear'] > 2014].copy()
movie = movie[movie['startYear'] < 2020].copy()
movie = movie[movie['numVotes'] > 200000].copy()
movie = movie[movie['genres'].str.find(r'\N') == -1].copy()




In [58]:
movie.dropna()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
354649,tt0369610,movie,Jurassic World,Jurassic World,0,2015,\N,124,"Action,Adventure,Sci-Fi",7,580591
420004,tt0437086,movie,Alita: Battle Angel,Alita: Battle Angel,0,2019,\N,122,"Action,Adventure,Sci-Fi",7.3,229473
430739,tt0448115,movie,Shazam!,Shazam!,0,2019,\N,132,"Action,Adventure,Comedy",7,257380
433811,tt0451279,movie,Wonder Woman,Wonder Woman,0,2017,\N,141,"Action,Adventure,Fantasy",7.4,571167
460755,tt0478970,movie,Ant-Man,Ant-Man,0,2015,\N,117,"Action,Adventure,Comedy",7.3,572603
...,...,...,...,...,...,...,...,...,...,...,...
6627009,tt7784604,movie,Hereditary,Hereditary,0,2018,\N,127,"Drama,Horror,Mystery",7.3,242613
6899944,tt8367814,movie,The Gentlemen,The Gentlemen,0,2019,\N,113,"Action,Comedy,Crime",7.8,243560
6999889,tt8579674,movie,1917,1917,0,2019,\N,119,"Drama,Thriller,War",8.3,432994
7088855,tt8772262,movie,Midsommar,Midsommar,0,2019,\N,148,"Drama,Horror,Mystery",7.1,203502


In [59]:
for i in range(len(movie)):
    if movie.iloc[i]['genres'].find(',') != -1:
        genre_lst = []
        genre_lst = movie.iloc[i]['genres'].split(',')
        new = {'tconst' : [], 'titleType' : [], 'primaryTitle' : [], 'startYear' : [], 
               'genres' : [], 'averageRating' : [], 'numVotes' : []}
        for j in range(len(genre_lst)):
            new['tconst'].append(movie['tconst'].iloc[i])
            new['titleType'].append('movie')
            new['primaryTitle'].append(movie['primaryTitle'].iloc[i])
            new['startYear'].append(movie['startYear'].iloc[i])
            new['genres'].append(genre_lst[j])
            new['averageRating'].append(movie['averageRating'].iloc[i])
            new['numVotes'].append(movie['numVotes'].iloc[i])
        new = pd.DataFrame(new)
        movie = pd.concat([movie, new])
        
movie = movie[movie['genres'].str.find(',') == -1]
movie

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
3233021,tt1396484,movie,It,It,0,2017,\N,135,Horror,7.3,461117
4930390,tt4034228,movie,Manchester by the Sea,Manchester by the Sea,0,2016,\N,137,Drama,7.8,248209
5349376,tt4975722,movie,Moonlight,Moonlight,0,2016,\N,111,Drama,7.4,274998
0,tt0369610,movie,Jurassic World,NaN,NaN,2015,NaN,NaN,Action,7,580591
1,tt0369610,movie,Jurassic World,NaN,NaN,2015,NaN,NaN,Adventure,7,580591
...,...,...,...,...,...,...,...,...,...,...,...
1,tt8772262,movie,Midsommar,NaN,NaN,2019,NaN,NaN,Horror,7.1,203502
2,tt8772262,movie,Midsommar,NaN,NaN,2019,NaN,NaN,Mystery,7.1,203502
0,tt8946378,movie,Knives Out,NaN,NaN,2019,NaN,NaN,Comedy,7.9,461679
1,tt8946378,movie,Knives Out,NaN,NaN,2019,NaN,NaN,Crime,7.9,461679


In [108]:
nodesT = pd.DataFrame()
nodesG = pd.DataFrame()
nodesT['id'] = movie['primaryTitle'].unique()
nodesG['id'] = movie['genres'].unique()
nodes = pd.concat([nodesT, nodesG])
groups = ['Movies' for i in range(len(nodesT['id']))] + ['Genre' for i in range(len(nodesG['id']))]
nodes['group'] = groups
nodes.to_csv('nodes.csv')

In [101]:
links = movie[['primaryTitle', 'genres']].copy()
links.rename(columns = {'genres' : 'source', 'primaryTitle' : 'target'}, inplace = True)
links.to_csv('links.csv')
links

,target,source
3233021,It,Horror
4930390,Manchester by the Sea,Drama
5349376,Moonlight,Drama
0,Jurassic World,Action
1,Jurassic World,Adventure
...,...,...
1,Midsommar,Horror
2,Midsommar,Mystery
0,Knives Out,Comedy
1,Knives Out,Crime


In [116]:
movie['count'] = [1 for i in range(len(movie))]
genreCount = movie[['genres', 'count']].groupby('genres').sum()
genreCount

,count
genres,
Action,74
Adventure,71
Animation,10
Biography,12
Comedy,36
Crime,23
Drama,65
Family,6
Fantasy,17


In [115]:
movie.to_csv('movie.csv')